# Importing Libraries

In [1]:
# libraries for automating the data analysis process

import os 
import io 
import sys
import msoffcrypto
from glob import glob

# library for parallel processing
# from concurrent.futures import ThreadPoolExecutor

# libraries for data analysis

import pandas as pd
import numpy as np

# libraries for automatic EDA

from ydata_profiling import ProfileReport

C:\DBA_Test\Code\sutd_eda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Defining Folder Variables

In [2]:
FILE_PASSWORD = '29012024'

# Set relative paths for different folders
MAIN_FOLDER = os.path.abspath(os.path.join(os.getcwd(), ".."))

CODE_FOLDER = os.path.join(MAIN_FOLDER, "Code")

PROFILE_FOLDER = os.path.join(CODE_FOLDER, "ProfileReports")

DATA_FOLDER = os.path.join(MAIN_FOLDER, "Data")

RAW_SHEETS_FOLDER = os.path.join(DATA_FOLDER, "Raw_Sheets")

CLEANED_SHEETS_FOLDER = os.path.join(DATA_FOLDER, "Cleaned_Sheets")


# Automatic Excel (.xlsx) parsing and loading

### Parallel Processing will not work with batch scripts

In [3]:
# def load_decrypt_excel(file_path):
#     """
#     Load and decrypt the Excel file using the given password.
#     Returns a decrypted workbook object.
#     FILE_PASSWORD is a global variable defined via bash script.
#     """
#     decrypted_workbook = io.BytesIO()
#     with open(file_path, 'rb') as file:
#         office_file = msoffcrypto.OfficeFile(file)
#         office_file.load_key(password=FILE_PASSWORD)
#         office_file.decrypt(decrypted_workbook)
#     return decrypted_workbook

# def separate_excel_to_csv(file_path):
#     """
#     Separate each sheet of the Excel file into individual CSV files.
#     """
#     decrypted_workbook = load_decrypt_excel(file_path)
#     df = pd.read_excel(decrypted_workbook, sheet_name=None)
#     for sheet, data in df.items():
#         csv_file_path = os.path.join(RAW_SHEETS_FOLDER, f"{sheet}.csv")
#         data.to_csv(csv_file_path, index=False)

# # Find all Excel files in the Data folder and its subdirectories
# excel_files = [file for file in glob(os.path.join(DATA_FOLDER, "**", "*.xlsx"), recursive=True)]

# # Parallelize processing using ThreadPoolExecutor, cuts down run time by half (tested)
# with ThreadPoolExecutor() as executor:
#     executor.map(separate_excel_to_csv, excel_files)

In [4]:
def load_decrypt_excel(file_path):
    """
    Load and decrypt the Excel file using the given password.
    Returns a decrypted workbook object.
    FILE_PASSWORD is a global variable defined via bash script.
    """
    decrypted_workbook = io.BytesIO()
    with open(file_path, 'rb') as file:
        office_file = msoffcrypto.OfficeFile(file)
        office_file.load_key(password=FILE_PASSWORD)
        office_file.decrypt(decrypted_workbook)
    return decrypted_workbook

def separate_excel_to_csv(file_path):
    """
    Separate each sheet of the Excel file into individual CSV files.
    """
    decrypted_workbook = load_decrypt_excel(file_path)
    df = pd.read_excel(decrypted_workbook, sheet_name=None)
    for sheet, data in df.items():
        csv_file_path = os.path.join(RAW_SHEETS_FOLDER, f"{sheet}.csv")
        data.to_csv(csv_file_path, index=False)

# Find all Excel files in the Data folder and its subdirectories
excel_files = [file for file in glob(os.path.join(DATA_FOLDER, "**", "*.xlsx"), recursive=True)]

# Iterate over each Excel file sequentially
for excel_file in excel_files:
    separate_excel_to_csv(excel_file)

# Defining Sheets Variables

In [5]:
# List all files in the Raw_Sheets folder
all_files = os.listdir(RAW_SHEETS_FOLDER)

# Filter only files that match the year naming convention (20XX.csv)
year_files = [file for file in all_files if file.startswith("20") and file.endswith(".csv")]

# Create variables for each year file, works with any number of years in the excel data
for year_file in year_files:
    year = year_file.split(".")[0]
    file_path = os.path.join(RAW_SHEETS_FOLDER, year_file)
    variable_name = f"FILEPATH_{year}"
    globals()[variable_name] = file_path

FILEPATH_PM = os.path.join(RAW_SHEETS_FOLDER, "No. of PM Available Daily.csv")

FILEPATH_DRIVER_DATA = os.path.join(RAW_SHEETS_FOLDER, "Drivers Data.csv")

SAVE_TO_CSV = os.path.join(CLEANED_SHEETS_FOLDER, "cleaned_sheet.csv")

SAVE_TO_HTML = os.path.join(PROFILE_FOLDER, "ydata_eda.html")

# Loading CSV files

In [6]:
# each year's filepaths are a variable in the global namespace

unmerged_2021 = pd.read_csv(FILEPATH_2021, encoding='utf-8')

unmerged_2022 = pd.read_csv(FILEPATH_2022, encoding='utf-8')

unmerged_2023 = pd.read_csv(FILEPATH_2023, encoding='utf-8')

unmerged_pm = pd.read_csv(FILEPATH_PM, encoding='ISO-8859-1')

unmerged_driver = pd.read_csv(FILEPATH_DRIVER_DATA, encoding='utf-8')

C:\Users\micha\AppData\Local\Temp\ipykernel_18000\3979976781.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  unmerged_2021 = pd.read_csv(FILEPATH_2021, encoding='utf-8')


C:\Users\micha\AppData\Local\Temp\ipykernel_18000\3979976781.py:5: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  unmerged_2022 = pd.read_csv(FILEPATH_2022, encoding='utf-8')


C:\Users\micha\AppData\Local\Temp\ipykernel_18000\3979976781.py:7: DtypeWarning: Columns (3,5,6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  unmerged_2023 = pd.read_csv(FILEPATH_2023, encoding='utf-8')


# Cleaning DateTime Entries

In [7]:
unmerged_2021['STARTTM'] = pd.to_datetime(unmerged_2021['STARTTM'], format='%Y-%m-%d %H:%M:%S')
unmerged_2021['ENDTM'] = pd.to_datetime(unmerged_2021['ENDTM'], format='%Y-%m-%d %H:%M:%S')

unmerged_2022['STARTTM'] = pd.to_datetime(unmerged_2022['STARTTM'])
unmerged_2022['ENDTM'] = pd.to_datetime(unmerged_2022['ENDTM'])

unmerged_2023['STARTTM'] = pd.to_datetime(unmerged_2023['STARTTM'])
unmerged_2023['ENDTM'] = pd.to_datetime(unmerged_2023['ENDTM'])

unmerged_pm['Date'] = pd.to_datetime(unmerged_pm['Date'])

# Merging all Years

In [8]:
merged_years = pd.concat([unmerged_2021, unmerged_2022, unmerged_2023], ignore_index=True)

merged_years.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,STARTTM,ENDTM,TripDuration,CTCOMPLETEDT,DriverId,PrimeMoverId,TrailerId,Status
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-07 08:00:00,2021-01-07 08:30:00,30.0,2021-01-07,2.0,215,1538.0,NaN
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-07 10:30:00,2021-01-07 11:00:00,30.0,2021-01-07,2.0,215,972.0,NaN
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-27 08:00:00,2021-01-27 08:30:00,30.0,2021-01-27,2.0,215,972.0,NaN
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-27 10:30:00,2021-01-27 11:00:00,30.0,2021-01-27,2.0,215,1538.0,NaN
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-11 08:00:00,2021-01-11 08:30:00,30.0,2021-01-11,2.0,215,1538.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299945,IWT,DUMMY,NONTRUCKING,NaN,0,NaN,NaN,IWT,42PTC,42PTC,2023-12-31 08:00:00,2023-12-31 17:00:00,NaN,2023-12-31,519.0,184,NaN,SHIFTING
1299946,IWT,DUMMY,NONTRUCKING,NaN,0,NaN,NaN,IWT,42PTC,42PTC,2023-12-31 08:00:00,2023-12-31 17:00:00,NaN,2023-12-31,570.0,103,NaN,SHIFTING
1299947,NONTRUCKINGJOB,DUMMY,NONTRUCKING,NaN,0,NaN,NaN,CHIEN,HB-CCK,HB-CCK,2023-12-31 08:30:00,2023-12-31 17:30:00,NaN,2023-12-31,580.0,220,NaN,OFF
1299948,NONTRUCKINGJOB,DUMMY,NONTRUCKING,NaN,0,NaN,NaN,CHIEN,HB-48P,HB-48P,2023-12-31 08:30:00,2023-12-31 17:30:00,NaN,2023-12-31,580.0,220,NaN,OFF


# Data Cleaning

## Dropping Non-Trucking Jobs

In [9]:
merged_years = merged_years[merged_years['JOBTYPE'] != 'NONTRUCKING']

## Cleaning PrimeMover IDs

### Replacing Non-numeric IDs (Years Sheet)

In [10]:
merged_years['PrimeMoverId'] = merged_years['PrimeMoverId'].replace('JACKSON', 1000)

merged_years['PrimeMoverId'] = merged_years['PrimeMoverId'].replace('XHSUBCON', 1001)

merged_years['PrimeMoverId'] = merged_years['PrimeMoverId'].replace('PTCLSUBCON', 1002)

merged_years['PrimeMoverId'] = merged_years['PrimeMoverId'].fillna(0).astype('int64')

### Replacing Non-numeric IDs (PM Sheet)

In [11]:
unmerged_pm['PrimeMoverId'] = unmerged_pm['PrimeMoverId'].fillna(0).astype('int64')

### Dropping Duplicated Prime Mover IDs from Asset Sheet

In [12]:
# # for each day and each PM ID, there should only be one unique entry
# unmerged_pm_dropped = unmerged_pm.drop_duplicates(subset=['PrimeMoverId', 'Date'], keep='first')

unmerged_pm_dropped = unmerged_pm[~((unmerged_pm[['Date', 'PrimeMoverId']].duplicated(keep='first')) & (unmerged_pm['Status'].isin(['NOINPUT', 'SUNDAY/HOLIDAY'])))]

In [13]:
unmerged_pm_dropped['Status'].unique()

array(['21', '17', 'SUNDAY/HOLIDAY', '26', 'SPARE', '13', 'OFF', '8',
       'WORKSHOP', '15', 'SHIFTING', '16', '10', '12', 'MEDICAL LEAVE',
       '14', '44', '6', '5', '22', '3', '9', '7', 'OTHER', '18', '23',
       '1', '4', 'ON LEAVE', '2', '11', 'URGENT LEAVE', '20', 'NOINPUT',
       '33', '28', '25', '43', '24', '30', '19', '50', '32', '60', '41',
       '27', '46', '29', '38', '36', '45', '42', '35', '48', '31', '34',
       '37', '51', '53', '49', '47', '39', '40', '52', '55', '59', '65',
       '54', '57', '56', '58', '64', '62'], dtype=object)

### Checking Unique Prime Mover IDs

In [14]:
merged_years['PrimeMoverId'].unique()

array([ 215,  259,  218,  145,  186,  225,  219,  143,  241,   22,  250,
        101,  232,  223,  187,  234,  309,  295,  305,  319,  317,  320,
        360,    1,  355,  103,  346,  364,  344,  363,  304,  316,  338,
        296,  139,  236,  217,  214,  188,  265,  351,   88,   69,  246,
        211,  237,  245,  201,  253,  220,  248,  233,  263,  252,  294,
        254,  221,  258,  240,  257,   64,  235,  204,   87,  144,   85,
        310,  332,  333,  342,  376,  347,  370,  341,  336,   13,  354,
        345,  365,  343,  362,  140,  340,   10,  297,  315,  374,  318,
        311,  298,  337,  307,  369,  313,  303,  339,  300,  334,  308,
        366,  306,  367,  312,  314,  299,  302,  373,  301,   98,   92,
        100,  107,  327,    2,  329,  359,  322,  325,  141,  323,  330,
        357,  321,  356,  324,  358,  350,  326,  331,  183,  194,  249,
        222,  260,  256,  266,  185,  231,  224,  228,  247,  255,  212,
         81,  239, 1000,  348,   17, 1001,   99,  1

### Number of Unique Prime Movers

In [15]:
merged_years['PrimeMoverId'].nunique()

224

### Sorted Array for Easier Visualisation (Years Sheet)

In [16]:
pm_total_year = merged_years['PrimeMoverId'].unique()
pm_total_year.sort()
print(pm_total_year)

[   1    2   10   13   17   22   64   69   81   85   87   88   92   98
   99  100  101  103  107  139  140  141  143  144  145  162  163  183
  184  185  186  187  188  192  194  201  204  211  212  214  215  217
  218  219  220  221  222  223  224  225  226  228  229  231  232  233
  234  235  236  237  238  239  240  241  245  246  247  248  249  250
  252  253  254  255  256  257  258  259  260  262  263  265  266  268
  270  271  274  275  292  294  295  296  297  298  299  300  301  302
  303  304  305  306  307  308  309  310  311  312  313  314  315  316
  317  318  319  320  321  322  323  324  325  326  327  328  329  330
  331  332  333  334  336  337  338  339  340  341  342  343  344  345
  346  347  348  350  351  354  355  356  357  358  359  360  362  363
  364  365  366  367  369  370  371  373  374  375  376  378  379  380
  381  382  383  384  385  386  387  388  389  391  392  393  394  395
  396  397  398  399  400  401  402  403  404  405  406  407  408  409
  410 

### Sorted Array for Easier Visualisation (PM Sheet)

In [17]:
pm_total = unmerged_pm_dropped['PrimeMoverId'].unique()
pm_total.sort()
print(pm_total)

[  0   1   2  10  13  17  22  81  87  88  92  97  99 100 101 102 103 111
 141 143 144 145 162 163 183 184 185 186 187 188 190 192 201 204 208 211
 212 213 214 215 217 218 219 220 221 222 223 224 225 226 231 232 233 234
 235 236 237 238 239 240 241 245 246 247 248 249 250 252 253 254 255 256
 257 258 259 260 263 265 266 268 269 270 271 273 274 275 276 289 292 293
 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311
 312 314 315 316 317 318 319 320 321 322 323 324 326 327 328 329 330 331
 332 333 334 336 337 338 339 340 341 342 344 345 346 347 354 355 356 357
 358 359 360 363 364 365 366 367 369 370 374 376 378 379 380 381 382 383
 384 385 386 387 392 393 394 395 396 397 398 399 403 404 405 406 407 418
 419 420 421 422 423 424 425 426 427 428 444 448 450 465 480 481 482 483
 484]


### Number of Unique Prime Movers in PM Sheet

In [18]:
unmerged_pm_dropped['PrimeMoverId'].nunique()

199

### Identifying Prime Mover IDs in Years Sheet that are not in PM Sheet

In [19]:
main_pm_list = np.setdiff1d(pm_total_year, pm_total)
print(main_pm_list)

[  64   69   85   98  107  139  140  194  228  229  262  313  325  343
  348  350  351  362  371  373  375  388  389  391  400  401  402  408
  409  410  411  412  416  417  475  477 1000 1001 1002]


## Cleaning Driver ID

### Converting Non-numeric to numeric Data

In [20]:
merged_years['DriverId'] = merged_years['DriverId'].fillna(0).astype('int64')

### Checking for duplicate driver IDs in Asset Sheet

In [21]:
unmerged_driver.drop_duplicates(subset='Driverid', keep='first', inplace=True)

### Show Unique Driver IDs (Years Sheet)

In [22]:
merged_years['DriverId'].unique()

array([  2,  29,  33,  37,  41,  51,  60,  67,  70,  72,  74,  75,  76,
        78,  82,  88, 104, 106, 108, 110, 113, 128, 139, 151, 152, 167,
       169, 171, 182, 185, 190, 201, 206, 208, 211, 227, 231, 250, 251,
       256, 260, 262, 264, 267, 272, 279, 282, 286, 287, 288, 291, 293,
       294, 295, 296, 305, 310, 311, 314, 323, 326, 334, 339, 563, 341,
       346, 349, 553, 355, 356, 360, 361, 362, 363, 367, 370, 372, 376,
       378, 379, 381, 384, 386, 394, 398, 400, 402, 403, 405, 407, 409,
       419, 427, 431, 433, 434, 437, 438, 445, 446, 447, 375, 396, 382,
       358, 345, 330, 318, 389, 432, 415, 444, 337, 422, 148, 277, 220,
       303, 255,  94,  62, 301, 176, 523, 213,  99, 302,  83,  66,  36,
        28,   0, 449, 450, 235, 222, 452, 453, 455,   4, 456, 454, 458,
       460, 461, 462, 172, 463, 474, 476, 475, 477, 478, 479, 486, 487,
       488, 489, 490, 491, 274, 421, 496, 492,  98, 499, 497, 498, 493,
       495, 494, 505, 503, 502, 504, 507, 506, 509, 511, 508, 19

### Show Unique Driver IDs (Drivers Sheet)

In [23]:
unmerged_driver['Driverid'].unique()

array([197, 455, 346,  60, 367, 448, 317, 148, 381, 323, 400, 297, 447,
       362, 377, 502, 419,  59, 389, 375, 314,  37, 295, 431,  98, 384,
       486, 262, 305, 227, 172, 361, 597, 326, 524, 405, 497, 573, 104,
       287, 211, 522, 571,  83, 411, 595, 151,  82, 222, 402, 255, 308,
       293, 505, 105, 461, 575, 398, 479, 394, 242, 264, 277, 476, 572,
       507, 496, 587, 449,  72, 563, 139,  79, 220,  66, 592, 493, 110,
       294, 376, 495, 518, 474, 257,  48, 260, 339, 274, 149, 429, 247,
       386, 506, 250, 191, 348, 388, 358,  28, 437, 363, 462, 360, 312,
       583, 477, 517, 525, 303, 306, 407, 584, 244,  89,  44, 167, 169,
       382, 579, 433, 450,  33, 120, 299, 370, 516, 324, 498, 452, 454,
       272, 296, 582, 586, 345, 288, 337, 409, 445, 523, 509, 379, 185,
        94, 310,  62, 499, 427, 456, 309, 152,  93, 313, 494,  35, 330,
       318, 267, 446, 213,  78, 438, 588, 176, 475, 341, 508, 304,  70,
       259, 190,  29, 334, 488, 504,  75, 434, 235, 520, 503,  3

# Merging Sheets

## Join by Driver ID

In [24]:
# validate many to one relationship, left join

merged_years_driver = pd.merge(merged_years, unmerged_driver, left_on='DriverId', right_on='Driverid', how='left', validate='m:1')

merged_years_driver.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,TrailerId,Status,Driverid,JOB CODE,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,1538.0,NaN,2.0,PRIME MOVER DRIVER,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,972.0,NaN,2.0,PRIME MOVER DRIVER,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,972.0,NaN,2.0,PRIME MOVER DRIVER,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,1538.0,NaN,2.0,PRIME MOVER DRIVER,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,1538.0,NaN,2.0,PRIME MOVER DRIVER,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,1934.0,NaN,356.0,PRIME MOVER DRIVER,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0
1222389,XI7007622,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,1265.0,NaN,169.0,PRIME MOVER DRIVER,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0
1222390,XI7007622,00-RE114,IMPORT,Double Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),WINGSENG,...,895.0,NaN,415.0,PRIME MOVER DRIVER,2017-08-14,NaN,31 yy:10 mm,MALAY,MALE,1.0
1222391,XI7007620,00-RE114,IMPORT,Single Mount,20,TK,LOADED,XINHUA,PSA/BT/KT,2 SERAYA P,...,1800.0,NaN,398.0,PRIME MOVER DRIVER,2015-11-02,NaN,40 yy:09 mm,CHINESE,MALE,1.0


## Cleaning CTCOMPLETEDT format

In [25]:
merged_years_driver['CTCOMPLETEDT'] = pd.to_datetime(merged_years_driver['CTCOMPLETEDT'], format='mixed', dayfirst=True)

## Join by Prime Mover ID

In [26]:
# validate many to one relationship, left join

merged_total = pd.merge(merged_years_driver, unmerged_pm_dropped, left_on=[merged_years_driver['PrimeMoverId'], merged_years_driver['CTCOMPLETEDT'].dt.date], right_on=[unmerged_pm_dropped['PrimeMoverId'], unmerged_pm_dropped['Date'].dt.date], validate='m:1', how='left')

## Dropping Intermediate Columns

In [27]:
merged_total = merged_total.drop(['key_0', 'key_1', 'PrimeMoverId_y', 'Status_x'], axis=1)

merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8
1222389,XI7007622,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-21,XINHUA,Team 1,8
1222390,XI7007622,00-RE114,IMPORT,Double Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),WINGSENG,...,2017-08-14,NaN,31 yy:10 mm,MALAY,MALE,1.0,2023-12-26,XINHUA,Team 1,7
1222391,XI7007620,00-RE114,IMPORT,Single Mount,20,TK,LOADED,XINHUA,PSA/BT/KT,2 SERAYA P,...,2015-11-02,NaN,40 yy:09 mm,CHINESE,MALE,1.0,2023-12-15,XINHUA,Team 1,8


# Cleaning Merged Sheet

## Drop rows where FROM and TO locations are identical

In [28]:
# boolean indexing to drop rows where 'FROMLOCSTR' is equal to 'TOLOCSTR'
merged_total = merged_total[merged_total['FROMLOCSTR'] != merged_total['TOLOCSTR']]

merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8
1222389,XI7007622,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-21,XINHUA,Team 1,8
1222390,XI7007622,00-RE114,IMPORT,Double Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),WINGSENG,...,2017-08-14,NaN,31 yy:10 mm,MALAY,MALE,1.0,2023-12-26,XINHUA,Team 1,7
1222391,XI7007620,00-RE114,IMPORT,Single Mount,20,TK,LOADED,XINHUA,PSA/BT/KT,2 SERAYA P,...,2015-11-02,NaN,40 yy:09 mm,CHINESE,MALE,1.0,2023-12-15,XINHUA,Team 1,8


## Removed Nontrucking Entries

In [29]:
merged_total= merged_total[merged_total['JOBTYPE'] != 'NONTRUCKING']

merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8
1222389,XI7007622,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-21,XINHUA,Team 1,8
1222390,XI7007622,00-RE114,IMPORT,Double Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),WINGSENG,...,2017-08-14,NaN,31 yy:10 mm,MALAY,MALE,1.0,2023-12-26,XINHUA,Team 1,7
1222391,XI7007620,00-RE114,IMPORT,Single Mount,20,TK,LOADED,XINHUA,PSA/BT/KT,2 SERAYA P,...,2015-11-02,NaN,40 yy:09 mm,CHINESE,MALE,1.0,2023-12-15,XINHUA,Team 1,8


## Status Type Conversion to Integer

In [30]:
merged_total.loc[:, 'Status_y'] = pd.to_numeric(merged_total['Status_y'], errors='coerce')
merged_total.loc[:, 'Status_y'].astype('Int64')

merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8.0
1222389,XI7007622,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-21,XINHUA,Team 1,8.0
1222390,XI7007622,00-RE114,IMPORT,Double Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),WINGSENG,...,2017-08-14,NaN,31 yy:10 mm,MALAY,MALE,1.0,2023-12-26,XINHUA,Team 1,7.0
1222391,XI7007620,00-RE114,IMPORT,Single Mount,20,TK,LOADED,XINHUA,PSA/BT/KT,2 SERAYA P,...,2015-11-02,NaN,40 yy:09 mm,CHINESE,MALE,1.0,2023-12-15,XINHUA,Team 1,8.0


## Dropping Irrelevant PrimeMover IDs from merged sheet

In [31]:
# removing prime movers in list 

removed_pm = [194, 228, 229, 262, 348, 350, 351, 371, 373, 375, 377, 388, 389, 391, 400, 401, 402, 408, 409, 410, 411, 412, 415, 416, 417, 475, 477, 486, 487]

merged_total = merged_total[~merged_total['PrimeMoverId_x'].isin(removed_pm)]

## Dropping Duplicate Rows

In [32]:
# merged_total= merged_total.drop_duplicates(keep='first', ignore_index=True)
merged_total = merged_total[~((merged_total.duplicated(keep='first')) & (merged_total['TypeOfTrip'].isin(['Double Mount'])))]

merged_total

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222397,TE8037442,00-RA186,EXPORT,Single Mount,20,GP,EMPTY,MAIN,PTC48(G),42PANDAN1,...,2008-03-17,NaN,51 yy:04 mm,CHINESE,MALE,1.0,2023-12-06,MAIN,FCL,34.0
1222398,TL9050450,00-RA186,LOCAL,Single Mount,20,TK,LOADED,CHIEN,COGENT TANK 15,48DRUMMING,...,2004-06-25,NaN,68 yy:03 mm,CHINESE,MALE,1.0,2023-12-04,CHIEN,CHIEN LI,8.0
1222399,TL9050450,00-RA186,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,48DRUMMING,STOLT NEL,...,2008-08-22,2023-05-10,39 yy:01 mm,CHINESE,MALE,0.0,2023-12-06,CHIEN,CHIEN LI,21.0
1222401,TE8037521,00-RA186,EXPORT,Single Mount,20,GP,LOADED,MAIN,42PANDAN1,PSA/PPT,...,2000-11-20,2022-10-02,58 yy:11 mm,CHINESE,MALE,0.0,2023-12-13,MAIN,FCL,8.0


# Feature Engineering

## Creating TripDuration Feature

In [33]:
# Calculate duration of each trip
merged_total['TripDuration'] = (merged_total['ENDTM'] - merged_total['STARTTM']).dt.total_seconds() / 3600  # Convert seconds to hours

# Display the DataFrame with the new column
merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,JOINED DATE,RESIGNED DATE,AGE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2007-06-07,2021-02-23,53 yy:07 mm,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222385,XI7007593,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,31GUL CR,14 PENJ RD,...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-12,XINHUA,Team 1,13.0
1222386,XI7007593,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,14 PENJ RD,EKY2,...,2007-12-01,NaN,55 yy:04 mm,CHINESE,MALE,1.0,2023-12-12,XINHUA,Team 1,13.0
1222387,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2014-11-01,NaN,55 yy:01 mm,MALAY,MALE,1.0,2023-12-12,XINHUA,Team 1,9.0
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2012-08-24,NaN,37 yy:05 mm,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8.0


## Cleaning Age Feature

In [34]:
# Extract ages in the format "53 yy:07mm"
merged_total.loc[:, 'years'] = merged_total['AGE'].str.extract(r'(\d+) yy').astype(float)
merged_total.loc[:, 'months'] = merged_total['AGE'].str.extract(r':(\d+) mm').astype(float)

# Extract ages in the format "41" and convert to float
merged_total.loc[:, 'single_age'] = merged_total['AGE'].str.extract(r'^(\d+)$').astype(float)

# Combine 'years' and 'months' into a single numerical value, rounded to 2 decimal places
merged_total.loc[:, 'AGE_Years'] = round(merged_total['years'] + merged_total['months'] / 12, 2)

# Fill NaN values in 'AGE_Years' column with values from 'single_age' column
merged_total['AGE_Years'] = merged_total['AGE_Years'].fillna(merged_total['single_age'])

# Drop intermediate columns 'years', 'months', 'AGE', and 'single_age'
merged_total.drop(['years', 'months', 'AGE', 'single_age'], axis=1, inplace=True)

## Creating Seniority Feature

In [35]:
# calculate seniority, setting resigned date to 31/12/2023 for those who have not resigned
merged_total['JOINED DATE'] = pd.to_datetime(merged_total['JOINED DATE'])
merged_total['RESIGNED DATE'] = pd.to_datetime(merged_total['RESIGNED DATE'], errors='coerce')
merged_total['RESIGNED DATE'] = merged_total['RESIGNED DATE'].fillna(pd.to_datetime("31/12/2023", format='%d/%m/%Y'))
merged_total['Seniority'] = ((merged_total['RESIGNED DATE'] - merged_total['JOINED DATE']).dt.days / 365.25).round(2)

# display data
merged_total.head(-10)

,OrderNumber,CUSTID,JOBTYPE,TypeOfTrip,CNTRSIZE,CNTRTYPE,LOAD_STATUS_CONT,Department,FROMLOCSTR,TOLOCSTR,...,RESIGNED DATE,RACE,GENDER,ACTIVE,Date,TOWHDDEPT,Team,Status_y,AGE_Years,Seniority
0,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2021-02-23,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0,53.58,13.72
1,TL9008453,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2021-02-23,CHINESE,MALE,0.0,2021-01-07,CHIEN,CHIEN LI,16.0,53.58,13.72
2,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2021-02-23,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0,53.58,13.72
3,TL9009105,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2021-02-23,CHINESE,MALE,0.0,2021-01-27,CHIEN,CHIEN LI,6.0,53.58,13.72
4,TL9008550,00-RE001,LOCAL,Single Mount,20,TK,EMPTY,CHIEN,JI DRUM,1 SERAYA AVE,...,2021-02-23,CHINESE,MALE,0.0,2021-01-11,CHIEN,CHIEN LI,13.0,53.58,13.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222385,XI7007593,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,31GUL CR,14 PENJ RD,...,2023-12-31,CHINESE,MALE,1.0,2023-12-12,XINHUA,Team 1,13.0,55.33,16.08
1222386,XI7007593,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,14 PENJ RD,EKY2,...,2023-12-31,CHINESE,MALE,1.0,2023-12-12,XINHUA,Team 1,13.0,55.33,16.08
1222387,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,LOADED,XINHUA,PSA/PPT,PTC ISL (G),...,2023-12-31,MALAY,MALE,1.0,2023-12-12,XINHUA,Team 1,9.0,55.08,9.16
1222388,XI7007610,00-RE114,IMPORT,Single Mount,20,GP,EMPTY,XINHUA,PTC ISL (G),CWT47JLNBUROH,...,2023-12-31,MALAY,MALE,1.0,2023-12-13,XINHUA,Team 1,8.0,37.42,11.35


# Renaming and Reorganising Features

In [36]:
merged_total_renamed = merged_total.rename(columns={'PrimeMoverId_x': 'PrimeMoverId', 'Status_y': 'Status'})

# dropping irrelevant columns
final_df = merged_total_renamed.drop(columns=['Date', 'GENDER', 'JOB CODE', 'ACTIVE', 'Driverid', 'TypeOfTrip', 'CNTRSIZE', 'CNTRTYPE', 'LOAD_STATUS_CONT', 'TOWHDDEPT'])

# final dataset still has some duplicated rows, possibly due to the left join 

final_df.head(-10)

,OrderNumber,CUSTID,JOBTYPE,Department,FROMLOCSTR,TOLOCSTR,STARTTM,ENDTM,TripDuration,CTCOMPLETEDT,DriverId,PrimeMoverId,TrailerId,JOINED DATE,RESIGNED DATE,RACE,Team,Status,AGE_Years,Seniority
0,TL9008453,00-RE001,LOCAL,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-07 08:00:00,2021-01-07 08:30:00,0.5,2021-01-07,2,215,1538.0,2007-06-07,2021-02-23,CHINESE,CHIEN LI,16.0,53.58,13.72
1,TL9008453,00-RE001,LOCAL,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-07 10:30:00,2021-01-07 11:00:00,0.5,2021-01-07,2,215,972.0,2007-06-07,2021-02-23,CHINESE,CHIEN LI,16.0,53.58,13.72
2,TL9009105,00-RE001,LOCAL,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-27 08:00:00,2021-01-27 08:30:00,0.5,2021-01-27,2,215,972.0,2007-06-07,2021-02-23,CHINESE,CHIEN LI,6.0,53.58,13.72
3,TL9009105,00-RE001,LOCAL,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-27 10:30:00,2021-01-27 11:00:00,0.5,2021-01-27,2,215,1538.0,2007-06-07,2021-02-23,CHINESE,CHIEN LI,6.0,53.58,13.72
4,TL9008550,00-RE001,LOCAL,CHIEN,JI DRUM,1 SERAYA AVE,2021-01-11 08:00:00,2021-01-11 08:30:00,0.5,2021-01-11,2,215,1538.0,2007-06-07,2021-02-23,CHINESE,CHIEN LI,13.0,53.58,13.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222385,XI7007593,00-RE114,IMPORT,XINHUA,31GUL CR,14 PENJ RD,2023-12-12 08:30:00,2023-12-12 09:30:00,1.0,2023-12-12,169,296,1771.0,2007-12-01,2023-12-31,CHINESE,Team 1,13.0,55.33,16.08
1222386,XI7007593,00-RE114,IMPORT,XINHUA,14 PENJ RD,EKY2,2023-12-12 09:30:00,2023-12-12 10:00:00,0.5,2023-12-12,169,296,1771.0,2007-12-01,2023-12-31,CHINESE,Team 1,13.0,55.33,16.08
1222387,XI7007610,00-RE114,IMPORT,XINHUA,PSA/PPT,PTC ISL (G),2023-12-12 09:00:00,2023-12-12 10:00:00,1.0,2023-12-12,379,338,1499.0,2014-11-01,2023-12-31,MALAY,Team 1,9.0,55.08,9.16
1222388,XI7007610,00-RE114,IMPORT,XINHUA,PTC ISL (G),CWT47JLNBUROH,2023-12-13 11:00:00,2023-12-13 12:00:00,1.0,2023-12-13,356,336,1934.0,2012-08-24,2023-12-31,MALAY,Team 1,8.0,37.42,11.35


# ydata-profiling automated EDA

In [37]:
profile = ProfileReport(final_df, title='Total PowerBI Data Profiling Report')

profile.to_file(SAVE_TO_HTML)

Summarize dataset:   0%|                                                  | 0/5 [00:00<?, ?it/s]

Summarize dataset:   0%|                      | 0/25 [00:08<?, ?it/s, Describe variable:JOBTYPE]

Summarize dataset:   4%|▌             | 1/25 [00:08<03:34,  8.92s/it, Describe variable:JOBTYPE]

Summarize dataset:   4%|▍          | 1/25 [00:09<03:34,  8.92s/it, Describe variable:Department]

Summarize dataset:   8%|▉          | 2/25 [00:09<01:35,  4.16s/it, Describe variable:Department]

...

Render HTML:   0%|                                                        | 0/1 [00:00<?, ?it/s]

Render HTML: 100%|████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]

Render HTML: 100%|████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]

Export report to file:   0%|                                              | 0/1 [00:00<?, ?it/s]

Export report to file: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 48.07it/s]

# Save to CSV File for Power BI Analysis

In [38]:
final_df.to_csv(SAVE_TO_CSV, index=False)